In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [2]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


E0000 00:00:1734941880.068369   37717 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734941880.071584   37717 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is configured properly.
Num GPUs Available: 1


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [4]:
f.predictions('amd')

I0000 00:00:1734941893.894533   37717 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1734941893.894669   37717 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14223 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1734941897.794786   37917 cuda_dnn.cc:529] Loaded cuDNN version 90300


AMD 5m Interval Timestamp: 2024-12-23 03:18:03


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.295969,0.305738,0.297455,0.183333,0.339394
prob_up,0.139731,0.142868,0.226214,0.0,0.0
prob_static,0.733534,0.631868,0.556672,0.875,1.0
prob_down,0.126735,0.225263,0.217114,0.125,0.0
precision,0.497121,0.504098,0.498182,0.416667,0.528139
recall,0.560606,0.532468,0.593074,0.519481,0.264069
f1,0.526958,0.517895,0.541502,0.462428,0.352092
support,"[461.0, 462.0, 462.0]","[461.0, 462.0, 462.0]","[461.0, 462.0, 462.0]","[461.0, 462.0, 462.0]","[461.0, 462.0, 462.0]"


AMD 15m Interval Timestamp: 2024-12-23 03:18:58


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,static,static,static
kelly_1:2.5,0.087407,0.080597,0.194771,0.130097,0.237624
prob_up,0.588634,0.588834,0.351003,0.375,0.0
prob_static,0.215453,0.231888,0.363465,0.625,1.0
prob_down,0.195913,0.179278,0.285532,0.0,0.0
precision,0.348148,0.343284,0.424837,0.378641,0.455446
recall,0.303226,0.296774,0.419355,0.503226,0.296774
f1,0.324138,0.318339,0.422078,0.432133,0.359375
support,"[155.0, 155.0, 154.0]","[155.0, 155.0, 154.0]","[155.0, 155.0, 154.0]","[155.0, 155.0, 154.0]","[155.0, 155.0, 154.0]"


...

,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,up
kelly_1:2.5,0.108943,0.086441,0.103279,0.041803,0.066667
prob_up,0.279651,0.272012,0.281735,0.25,1.0
prob_static,0.456856,0.467832,0.394204,0.5,0.0
prob_down,0.263493,0.260156,0.324061,0.25,0.0
precision,0.363531,0.347458,0.359485,0.315574,0.333333
recall,0.416223,0.381649,0.408245,0.409574,1.0
f1,0.388097,0.363752,0.382316,0.356481,0.5
support,"[751.0, 752.0, 750.0]","[751.0, 752.0, 750.0]","[751.0, 752.0, 750.0]","[751.0, 752.0, 750.0]","[751.0, 752.0, 750.0]"


AMD 1wk Interval Timestamp: 2024-12-23 03:21:32


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,static,down
kelly_1:2.5,0.069364,0.06375,0.086585,0.109091,0.068683
prob_up,0.261903,0.108361,0.381261,0.125,0.0
prob_static,0.182803,0.068033,0.203121,0.5,0.0
prob_down,0.555294,0.823607,0.415618,0.375,1.0
precision,0.33526,0.33125,0.347561,0.363636,0.334773
recall,0.374194,0.341935,0.367742,0.441558,1.0
f1,0.353659,0.336508,0.357367,0.398827,0.501618
support,"[154.0, 154.0, 155.0]","[154.0, 154.0, 155.0]","[154.0, 154.0, 155.0]","[154.0, 154.0, 155.0]","[154.0, 154.0, 155.0]"


AMD 1mo Interval Timestamp: 2024-12-23 03:22:00


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,up,down
kelly_1:2.5,-0.031579,0.006452,-0.00625,0.030769,0.056522
prob_up,0.766421,0.564107,0.496662,0.625,0.0
prob_static,0.071137,0.055935,0.16534,0.375,0.0
prob_down,0.162442,0.379958,0.337998,0.0,1.0
precision,0.263158,0.290323,0.28125,0.307692,0.326087
recall,0.322581,0.290323,0.290323,0.258065,1.0
f1,0.289855,0.290323,0.285714,0.280702,0.491803
support,"[31.0, 31.0, 30.0]","[31.0, 31.0, 30.0]","[31.0, 31.0, 30.0]","[31.0, 31.0, 30.0]","[31.0, 31.0, 30.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')